In [51]:
import pandas as pd
import re
import numpy as np
import gc
from fuzzywuzzy import fuzz

In [52]:
df_contribuion = pd.read_csv('data/contribution_processed.csv')
df_companies = pd.read_csv('data/ma-companies-on-linkedin.csv', sep = ';')

In [53]:
companies_list = df_companies['Company name'].unique()

In [54]:
employer_list = df_contribuion['Employer'].unique()

In [57]:
def clean_text(text):
    '''
    Remove special charecters, numbers
    Return lower case alphabet charecters only
    '''
    text = str(text).lower()
    text = re.sub('[^A-Za-z]+', '', text)
    return text

In [58]:
companies_list_clean = np.array([clean_text(x) for x in companies_list.copy()])

In [59]:
employer_list_clean = np.array([clean_text(x) for x in employer_list.copy()])

In [37]:
# MATCHING

In [81]:
match = []
companies_list_clean = companies_list_clean.astype('O')
employer_list_clean = employer_list_clean.astype('O')

for employer in employer_list_clean:
    for company in companies_list_clean:
        if fuzz.ratio(employer, company) >= 85:
            match.append(employer)

In [83]:
match_set = set(match)

In [88]:
df_contribuion['Employer_clean'] = df_contribuion['Employer'].apply(clean_text)

In [93]:
df_companies['Company_clean'] = df_companies['Company name'].apply(clean_text)

In [89]:
def company_search(name):
    return name in match_set

In [90]:
df_contribuion['Industry_new'] = df_contribuion['Employer_clean'].apply(company_search)

In [92]:
df_contribuion['Industry_new'].value_counts()

False    95910
True     22551
Name: Industry_new, dtype: int64

In [116]:
match_ind = {}
for com in match_set:
    industry = df_companies[df_companies['Company_clean'] == com]['Industry'].values
    if len(industry) == 0:
        continue
    match_ind[com] = industry[0]

In [124]:
employer_dict = {}
for employer in match_set:
    for company in companies_list_clean:
        if fuzz.ratio(employer, company) >= 85:
            employer_dict[employer] = company

In [131]:
employer_dict_list = list(employer_dict.keys())
company_dict_list = list(employer_dict.values())

In [145]:
def com_search(name):
    if name in employer_dict.keys():
        industry = df_companies[df_companies['Company_clean'] == employer_dict[name]]['Industry'].values[0]
        return industry

In [146]:
df_contribuion['Industry_new'] = df_contribuion['Employer_clean'].apply(com_search)

In [158]:
df_contribuion.drop(columns=['Employer_clean'], inplace=True)

In [161]:
df_contribuion.head()

,Unnamed: 0,Unnamed: 0.1,Address,Amount,City,Contributor_ID,Contributor_Type,Date,Employer,First_Name,...,Occupation,Principal_Officer,Report_ID,State,Zip,Donor_Name,Industry,PAC,Lobbyist,Industry_new
0,0,7373,270 S. Common Street,500.0,Lynn,NaN,Individual,2015-01-01,McGrath Enterprises,Patrick,...,Developer,NaN,217037,MA,01905,PATRICK MCGRATH,NaN,NaN,False,None
1,1,7374,186 Perkins Row,250.0,Topsfield,NaN,Individual,2015-01-15,Self,Thomas,...,Attorney,NaN,217037,MA,01983,THOMAS DEMAKIS,Healthcare,NaN,False,professional training & coaching
2,2,7634,11 Lake Ave.,25.0,Lynn,NaN,Individual,2015-03-02,NaN,Loretta,...,NaN,NaN,217124,MA,10904,LORETTA O'DONNELL,NaN,NaN,False,None
3,3,7635,7 William St.,50.0,Lynn,NaN,Individual,2015-03-02,NaN,Stephen,...,NaN,NaN,217124,MA,01904,STEPHEN SPENCER,NaN,NaN,False,None
4,4,7636,20 Bulfinch Ter.,50.0,Lynn,NaN,Individual,2015-03-02,NaN,Ann Marie,...,NaN,NaN,217124,MA,01902,ANN MARIE LEONARD,NaN,NaN,False,None


In [162]:
df_contribuion.to_csv('data/contribution_experiment.csv')